-sandbox

<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning" style="width: 600px">
</div>

# Revenue by Traffic Lab
Get the 3 traffic sources generating the highest total revenue.
1. Aggregate revenue by traffic source
2. Get top 3 traffic sources by total revenue
3. Clean revenue columns to have two decimal places

##### Methods
- <a href="https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.DataFrame.html" target="_blank">DataFrame</a>: **`groupBy`**, **`sort`**, **`limit`**
- <a href="https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.Column.html?highlight=column#pyspark.sql.Column" target="_blank">Column</a>: **`alias`**, **`desc`**, **`cast`**, **`operators`**
- <a href="https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql.html?#functions" target="_blank">Built-in Functions</a>: **`avg`**, **`sum`**

In [0]:
%run ../Includes/Classroom-Setup

Deleted the working directory dbfs:/user/si@elisaandgeeks.com/dbacademy/aspwd/asp_3_1l_revenue_by_traffic_lab


Your working directory is
dbfs:/user/si@elisaandgeeks.com/dbacademy/aspwd

The source for this dataset is
wasbs://courseware@dbacademy.blob.core.windows.net/apache-spark-programming-with-databricks/v02/

Skipping install of existing dataset to
dbfs:/user/si@elisaandgeeks.com/dbacademy/aspwd/datasets


Out[5]: DataFrame[key: string, value: string]

### Setup
Run the cell below to create the starting DataFrame **`df`**.

In [0]:
from pyspark.sql.functions import col

# Purchase events logged on the BedBricks website
df = (spark.read.format("delta").load(events_path)
      .withColumn("revenue", col("ecommerce.purchase_revenue_in_usd"))
      .filter(col("revenue").isNotNull())
      .drop("event_name")
     )

#display(df)

### 1. Aggregate revenue by traffic source
- Group by **`traffic_source`**
- Get sum of **`revenue`** as **`total_rev`**
- Get average of **`revenue`** as **`avg_rev`**

Remember to import any necessary built-in functions.

In [0]:
# TODO
from pyspark.sql.functions import avg, col, sum

traffic_df = (df
              .groupBy("traffic_source")
              .agg(sum(col("revenue")).alias("total_rev"),
                  avg(col("revenue")).alias("avg_rev"))
)

display(traffic_df)

traffic_source,total_rev,avg_rev
instagram,1.6177893E7,1083.437784623627
direct,1.270456E7,1083.175036234973
youtube,8044326.0,1087.2180024327613
email,7.880000029999994E7,983.2915347084434
facebook,2.4797837E7,1076.6221074111058
google,4.7218429E7,1086.8302950789487


**1.1: CHECK YOUR WORK**

In [0]:
from pyspark.sql.functions import round

expected1 = [(12704560.0, 1083.175), (78800000.3, 983.2915), (24797837.0, 1076.6221), (47218429.0, 1086.8303), (16177893.0, 1083.4378), (8044326.0, 1087.218)]
test_df = traffic_df.sort("traffic_source").select(round("total_rev", 4).alias("total_rev"), round("avg_rev", 4).alias("avg_rev"))
result1 = [(row.total_rev, row.avg_rev) for row in test_df.collect()]

assert(expected1 == result1)
print("All test pass")

All test pass


### 2. Get top three traffic sources by total revenue
- Sort by **`total_rev`** in descending order
- Limit to first three rows

In [0]:
# TODO
top_traffic_df = (traffic_df.orderBy("total_rev", ascending=False).limit(3))
display(top_traffic_df)

traffic_source,total_rev,avg_rev
email,7.880000029999994E7,983.2915347084434
google,4.7218429E7,1086.8302950789487
facebook,2.4797837E7,1076.6221074111058


**2.1: CHECK YOUR WORK**

In [0]:
expected2 = [(78800000.3, 983.2915), (47218429.0, 1086.8303), (24797837.0, 1076.6221)]
test_df = top_traffic_df.select(round("total_rev", 4).alias("total_rev"), round("avg_rev", 4).alias("avg_rev"))
result2 = [(row.total_rev, row.avg_rev) for row in test_df.collect()]

assert(expected2 == result2)
print("All test pass")

All test pass


### 3. Limit revenue columns to two decimal places
- Modify columns **`avg_rev`** and **`total_rev`** to contain numbers with two decimal places
  - Use **`withColumn()`** with the same names to replace these columns
  - To limit to two decimal places, multiply each column by 100, cast to long, and then divide by 100

In [0]:
# TODO
final_df = (top_traffic_df
            .withColumn("avg_rev", (col("avg_rev")*100).cast("long")/100)
            .withColumn("total_rev", (col("total_rev")*100).cast("long")/100)
)

display(final_df)

traffic_source,total_rev,avg_rev
email,7.880000029E7,983.29
google,4.7218429E7,1086.83
facebook,2.4797837E7,1076.62


**3.1: CHECK YOUR WORK**

In [0]:
expected3 = [(78800000.29, 983.29), (47218429.0, 1086.83), (24797837.0, 1076.62)]
result3 = [(row.total_rev, row.avg_rev) for row in final_df.collect()]

assert(expected3 == result3)
print("All test pass")

All test pass


### 4. Bonus: Rewrite using a built-in math function
Find a built-in math function that rounds to a specified number of decimal places

In [0]:
# TODO
from pyspark.sql.functions import round

bonus_df = (top_traffic_df
            .withColumn("avg_rev", round("avg_rev",2))
            .withColumn("total_rev", round("total_rev",2))
)

display(bonus_df)

traffic_source,total_rev,avg_rev
email,7.88000003E7,983.29
google,4.7218429E7,1086.83
facebook,2.4797837E7,1076.62


**4.1: CHECK YOUR WORK**

In [0]:
expected4 = [(78800000.3, 983.29), (47218429.0, 1086.83), (24797837.0, 1076.62)]
result4 = [(row.total_rev, row.avg_rev) for row in bonus_df.collect()]

assert(expected4 == result4)
print("All test pass")

All test pass


### 5. Chain all the steps above

In [0]:
# TODO
chain_df = (df
            .groupBy("traffic_source")
            .agg(sum(col("revenue")).alias("total_rev"),
                 avg(col("revenue")).alias("avg_rev"))
            .orderBy("total_rev", ascending=False).limit(3)
            .withColumn("avg_rev", round("avg_rev",2))
            .withColumn("total_rev", round("total_rev",2))
)

display(chain_df)

traffic_source,total_rev,avg_rev
email,7.88000003E7,983.29
google,4.7218429E7,1086.83
facebook,2.4797837E7,1076.62


**5.1: CHECK YOUR WORK**

In [0]:
expected5 = [(78800000.3, 983.29), (47218429.0, 1086.83), (24797837.0, 1076.62)]
result5 = [(row.total_rev, row.avg_rev) for row in chain_df.collect()]

assert(expected5 == result5)
print("All test pass")

All test pass


### Clean up classroom

In [0]:
classroom_cleanup()

-sandbox
&copy; 2022 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the <a href="https://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/>
<a href="https://databricks.com/privacy-policy">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use">Terms of Use</a> | <a href="https://help.databricks.com/">Support</a>